## Task 1: Soduku Solver

In [4]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        # List of all cell coordinates
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        # Initialize domains: if a cell is pre-filled, its domain is that number; else, 1-9.
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        # Generate a neighbor mapping for forward checking (all cells sharing a row, column, or subgrid)
        self.constraints = self.generate_constraints()

    def generate_constraints(self):
        """
        For each cell (variable), generate a set of all neighboring cells that share a row, column,
        or 3x3 subgrid.
        """
        constraints = {}
        for i in range(9):
            for j in range(9):
                neighbors = set()
                # Same row
                for col in range(9):
                    if col != j:
                        neighbors.add((i, col))
                # Same column
                for row in range(9):
                    if row != i:
                        neighbors.add((row, j))
                # Same 3x3 subgrid
                start_row, start_col = (i // 3) * 3, (j // 3) * 3
                for r in range(start_row, start_row + 3):
                    for c in range(start_col, start_col + 3):
                        if (r, c) != (i, j):
                            neighbors.add((r, c))
                constraints[(i, j)] = neighbors
        return constraints

    def is_valid(self, var, value):
        """
        Check if assigning 'value' to the cell at position 'var' violates any constraints:
        row, column, or subgrid uniqueness.
        """
        i, j = var
        # Check the row
        for col in range(9):
            if self.grid[i][col] == value:
                return False
        # Check the column
        for row in range(9):
            if self.grid[row][j] == value:
                return False
        # Check the 3x3 subgrid
        start_row, start_col = (i // 3) * 3, (j // 3) * 3
        for r in range(start_row, start_row + 3):
            for c in range(start_col, start_col + 3):
                if self.grid[r][c] == value:
                    return False
        return True

    def forward_checking(self, var, value):
        """
        After assigning 'value' to 'var', remove that value from the domains of all neighboring
        unassigned cells. Return a list of (neighbor, value) pairs that were removed so they can be
        restored during backtracking.
        """
        affected = []
        for neighbor in self.constraints[var]:
            r, c = neighbor
            # Consider only unassigned cells
            if self.grid[r][c] == 0 and value in self.domains[neighbor]:
                self.domains[neighbor].remove(value)
                affected.append(neighbor)
        return affected

    def restore_domains(self, affected, value):
        """
        Restore the 'value' back into the domains of all affected neighbor cells.
        """
        for var in affected:
            self.domains[var].add(value)

    def backtrack(self):
        """
        Recursive backtracking search using MRV heuristic and forward checking.
        """
        # Check if assignment is complete (i.e. no cell is 0)
        if all(self.grid[i][j] != 0 for i in range(9) for j in range(9)):
            return True

        # Select the next unassigned variable using MRV (minimum remaining values)
        unassigned_vars = [(i, j) for i in range(9) for j in range(9) if self.grid[i][j] == 0]
        var = min(unassigned_vars, key=lambda v: len(self.domains[v]))
        i, j = var

        # Try each value in the current domain (sorted order for consistency)
        for value in sorted(self.domains[var]):
            if self.is_valid(var, value):
                # Assign the value and update domain for this cell
                self.grid[i][j] = value
                original_domain = self.domains[var].copy()
                self.domains[var] = {value}

                # Apply forward checking to update neighbors' domains
                affected = self.forward_checking(var, value)
                # If any neighbor's domain becomes empty, this assignment fails.
                failure = any(self.grid[r][c] == 0 and len(self.domains[(r, c)]) == 0 for (r, c) in self.constraints[var])
                if not failure:
                    if self.backtrack():
                        return True

                # Backtrack: undo the assignment and restore domains
                self.grid[i][j] = 0
                self.domains[var] = original_domain
                self.restore_domains(affected, value)

        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None


# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
